# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:

# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht
import hebrew_tokenizer as ht

c:\Users\haim5\OneDrive\שולחן העבודה\מדמח\שנה ב\סמסטר קיץ\למידת מכונה\מטלות\Matala 3\assignment3-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
def clean_data(df: pd.DataFrame):
    # Removing non Hebrew charecters
    df['story'] = df['story'].str.replace(r'[^\u0590-\u05fe\s]+', '', regex=True)

    # STEEMING:
        # Remove prefixes : 'כ' 'ו' 'ב' 'ש' 'ה' 'ל' 'כש' 'לכש' 'וש' 'וכש'
    df['story'] = df['story'].apply(lambda x: re.sub(r'\b(כ|ו|כש|וכש|וש|לכש|ב|ש|ל)', '', x))
        # Remove suffixes : 'תי' 'נו' 'ים' 'ו'
    df['story'] = df['story'].apply(lambda x: re.sub(r'(תי|ו|נו|ים)\b', '', x))
    df['story'] = df['story'].apply(lambda x: re.sub(r'ות\b', 'ה', x)) # בחורות -> בחורה
    
    # Removing words that has less than 3 letters - many of them are stop words
    df['story'] = df['story'].str.replace(r'\b\w{1,2}\b', '', regex=True)     
    
    # Removing multiply spaces
    df['story'] = df['story'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [12]:
clean_data(df_train)
clean_data(df_test)

In [13]:
X = df_train['story']
y = df_train['gender']
y = y.replace('m', 1)
y = y.replace('f', 0)

In [14]:
print(X.shape)
print(y.shape)

(753,)
(753,)


## TEST

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

In [16]:
models = [
    {'name': 'Logistic Regression', 'estimator': LogisticRegression(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000, 2500], 'clf__C': [0.01, 0.1, 1, 10], 'clf__class_weight': [None, 'balanced']}},
    {'name': 'Random Forest', 'estimator': RandomForestClassifier(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000], 'clf__n_estimators': [50, 100, 150], 'clf__class_weight': [None, 'balanced']}},
    {'name': 'Decision Tree', 'estimator': DecisionTreeClassifier(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000, 2500], 'clf__max_depth': [5, 10, 15, 20], 'clf__class_weight': [None, 'balanced']}},
    {'name': 'K-Nearest Neighbors', 'estimator': KNeighborsClassifier(), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000], 'clf__n_neighbors': [3, 5, 7]}},
    {'name': 'Support Vector Machine', 'estimator': SVC(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000], 'clf__C': [0.1, 1, 10], 'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'clf__class_weight': [None, 'balanced']}},
    {'name': 'Multinomial Naive Bayes', 'estimator': MultinomialNB(), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000, 2200], 'clf__alpha': [0.01, 0.1, 0.5, 1]}},
    {'name': 'Linear Support Vector Classification', 'estimator': LinearSVC(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000], 'clf__C': [0.1, 1, 10]}},
    {'name': 'Stochastic Gradient Descent', 'estimator': SGDClassifier(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000], 'clf__alpha': [0.0001, 0.001, 0.01], 'clf__max_iter': [500, 1000, 1500]}},
    {'name': 'Perceptron', 'estimator': Perceptron(random_state=42), 'hyperparameters': {'vect__max_features': [1500, 1750, 2000], 'clf__alpha': [0.0001, 0.001, 0.01], 'clf__max_iter': [500, 1000, 1500]}},
]

In [17]:
def f1Avg(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

In [18]:
#we will check how much the word appears and if its rare
vectorizers = [CountVectorizer(ngram_range=(1,1)), TfidfVectorizer(ngram_range=(1,1)),
              CountVectorizer(ngram_range=(1, 2)), TfidfVectorizer(ngram_range=(1, 2)),
              CountVectorizer(ngram_range=(1, 3)), TfidfVectorizer(ngram_range=(1, 3)),
              CountVectorizer(ngram_range=(1,6)), TfidfVectorizer(ngram_range=(1,6))
              ]

In [19]:
best_model = None
best_score = 0

for vect in vectorizers:
    print('===================================================================================')
    print("Vectorizer: %s\n" % vect.__class__.__name__)
    for model in models:
        print("Model: %s" % model['name'])
        pipeline = Pipeline([
            ('vect', vect),
            ('clf', model['estimator'])
        ])

        grid_search = GridSearchCV(pipeline, model['hyperparameters'], cv=10, scoring=make_scorer(f1Avg))
        grid_search.fit(X, y)

        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = grid_search.best_estimator_
            best_params=grid_search.best_params_
            
        print("Best hyperparameters: %s" % grid_search.best_params_)
        print("Cross-validation score: %0.2f" % grid_search.best_score_)
        print('===================================================================================')
        
print("The best model is:")
print(best_model)

Vectorizer: CountVectorizer

Model: Logistic Regression


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=42))]),
             param_grid={'clf__C': [0.01, 0.1, 1, 10],
                         'clf__class_weight': [None, 'balanced'],
                         'vect__max_features': [1500, 1750, 2000, 2500]},
             scoring=make_scorer(f1Avg))

Best hyperparameters: {'clf__C': 0.1, 'clf__class_weight': 'balanced', 'vect__max_features': 2500}
Cross-validation score: 0.68
Model: Random Forest


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'clf__class_weight': [None, 'balanced'],
                         'clf__n_estimators': [50, 100, 150],
                         'vect__max_features': [1500, 1750, 2000]},
             scoring=make_scorer(f1Avg))

Best hyperparameters: {'clf__class_weight': None, 'clf__n_estimators': 50, 'vect__max_features': 2000}
Cross-validation score: 0.53
Model: Decision Tree


...

Best hyperparameters: {'clf__C': 10, 'vect__max_features': 1750}
Cross-validation score: 0.65
Model: Stochastic Gradient Descent


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(ngram_range=(1, 6))),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01],
                         'clf__max_iter': [500, 1000, 1500],
                         'vect__max_features': [1500, 1750, 2000]},
             scoring=make_scorer(f1Avg))

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__max_iter': 500, 'vect__max_features': 1750}
Cross-validation score: 0.66
Model: Perceptron


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(ngram_range=(1, 6))),
                                       ('clf', Perceptron(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01],
                         'clf__max_iter': [500, 1000, 1500],
                         'vect__max_features': [1500, 1750, 2000]},
             scoring=make_scorer(f1Avg))

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__max_iter': 500, 'vect__max_features': 1500}
Cross-validation score: 0.66
The best model is:
Pipeline(steps=[('vect',
                 TfidfVectorizer(max_features=2500, ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(C=1, class_weight='balanced',
                                    random_state=42))])


In [21]:
model=best_model
model

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_features=2500, ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(C=1, class_weight='balanced',
                                    random_state=42))])

In [22]:
best_params

{'clf__C': 1, 'clf__class_weight': 'balanced', 'vect__max_features': 2500}

In [23]:
best_score

0.7234804052067407

### Predict

In [24]:
df_predicted  = df_test.copy()

In [25]:
model.fit(df_train['story'], df_train['gender'])

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_features=2500, ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(C=1, class_weight='balanced',
                                    random_state=42))])

In [26]:
df_predicted['predicted_category'] = model.predict(df_test['story'])
df_predicted.drop(['story',], axis=1, inplace=True)
df_predicted.predicted_category.replace({0:'f',1:'m'},inplace=True)

In [27]:
df_predicted.head()

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


In [28]:
df_predicted.tail()

,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


In [29]:
df_predicted['predicted_category'].unique()

array(['m', 'f'], dtype=object)

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [30]:
df_predicted.to_csv('classification_results.csv',index=False)

In [31]:
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m
